# NYT Spelling Bee Solver

Enter the Spelling Bee letters when prompted. The first letter is the required letter and must appear in every answer. The solver uses `nytbee_dict.txt` as the word list and prints hints in a format similar to the NYT Spelling Bee hint page.


In [ ]:
from __future__ import annotations

from collections import defaultdict
from pathlib import Path


def load_words(path: Path) -> list[str]:
    if not path.exists():
        raise FileNotFoundError(f"Missing wordlist: {path}")
    words = [line.strip().lower() for line in path.read_text().splitlines()]
    return [word for word in words if word.isalpha()]


def solve_spelling_bee(letters: str, wordlist_path: Path) -> tuple[list[str], list[str], str]:
    cleaned = ''.join(char for char in letters.lower() if char.isalpha())
    if len(cleaned) < 2:
        raise ValueError("Please provide at least two letters.")

    required = cleaned[0]
    allowed = set(cleaned)

    words = [
        word
        for word in load_words(wordlist_path)
        if len(word) >= 4 and required in word and set(word) <= allowed
    ]
    pangrams = [word for word in words if set(cleaned) <= set(word)]
    return sorted(words), sorted(pangrams), cleaned


def print_hint_page(words: list[str], pangrams: list[str], letters: str) -> None:
    print('NYT Spelling Bee Hint Page')
    print('-' * 30)
    print(f"Letters: {', '.join(letters)} (required: {letters[0]})")
    print(f"Total words: {len(words)}")
    print(f"Pangrams ({len(pangrams)}): {', '.join(pangrams) if pangrams else 'None'}")

    grouped: dict[int, list[str]] = defaultdict(list)
    for word in words:
        grouped[len(word)].append(word)

    for length in sorted(grouped):
        word_list = ', '.join(sorted(grouped[length]))
        print(f"{length} letters ({len(grouped[length])}): {word_list}")


In [ ]:
letters_input = input('Enter the Spelling Bee letters (required letter first): ')
words, pangrams, cleaned_letters = solve_spelling_bee(letters_input, Path('nytbee_dict.txt'))
print_hint_page(words, pangrams, cleaned_letters)
